# Kirisu 2

#### Simple editor for stopped-flow and other spectra

by *Sergio Boneta Martínez*  
based on the original **kirisu** by *Jose Ramon Peregrina* and *Jorge Estrada*

GPLv3 (C) 2022 @ *Universidad de Zaragoza*

In [ ]:
%%capture

#@title Prepare the environment
#@markdown Execute this cell before running the following cells. Only need to be done once, the rest can be run multiple times.

import os
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from google.colab import files


class TDSpectrum:
    '''
        Time-Dependent Spectrum

        Parameters
        ----------
        filename : str
            path to spectrum file to read

        Attributes
        ----------
        filename : str
            basename of spectrum file readed
        times : ndarray(n)
            array of times
        lambdas : ndarray(m)
            array of wavelengths
        absorb : ndarray(m,n)
            matrix of absorbances

        Properties
        ----------
        n_spec : int
            number of spectra
        n_times : int
            number of times
        n_lambdas : int
            number of wavelengths
        filename_trim : str
            filename with '_t' appended to the basename
    '''

    def __init__(self, filename=None):
        self.times = np.array([])
        self.lambdas = np.array([])
        self.absorb = np.array([])
        self.filename = ""
        if filename is not None:
            self.read(filename)

    @property
    def n_spec(self) -> int:
        return self.n_times

    @property
    def n_times(self) -> int:
        return self.times.shape[0]

    @property
    def n_lambdas(self) -> int:
        return self.lambdas.shape[0]

    @property
    def filename_trim(self) -> str:
        return "{}_t{}".format(*os.path.splitext(self.filename))

    def read(self, filename, format=None) -> None:
        '''
            Wrapper to read files based on extension/format

            If 'format' is not specified, the extension of the file
            is used. If the extension is not recognized, the file
            is assumed to be in the 'glb' format.

            Supported formats: glb
        '''
        # guess format from extension if not specified
        readers = {
            'glb' : self.read_glb
            }
        # check if file exists
        if not os.path.exists(filename):
            raise FileNotFoundError(f'File not found: {filename}')
        # assign format based on input argument/file extension/default
        extension = os.path.splitext(filename)[1][1:].lower()
        if format is not None:
            if format.lower() in readers:
                format = format.lower()
            else:
                raise ValueError(f'Unknown format to read: {format}')
        elif extension in readers:
            format = extension
        else:
            format = list(readers.keys())[0]
        # read file based on format
        readers[format](filename)

    def read_glb(self, filename) -> None:
        '''Read a spectrum from a GLB file'''
        def _find_ndx(l, v):
            '''Find index of firs occurence of value v in list l'''
            return [i for i, x in enumerate(l) if x.lower().startswith(v)][0]
        # read file
        self.filename = os.path.basename(filename)
        with open(filename, 'r') as f:
            data = [line.strip() for line in f.readlines() if line.strip()]
        # get dimensions
        n_spec = int(data[_find_ndx(data, 'n_spec')].split()[1])
        n_lamb = int(data[_find_ndx(data, 'n_lam')].split()[1])
        # times
        time_ndx = int(_find_ndx(data, 'times:')) + 1
        self.times = np.array(data[time_ndx:time_ndx+n_spec], dtype=float)
        # wavelengths
        lamb_ndx = int(_find_ndx(data, 'lambda:')) + 1
        self.lambdas = np.array(data[lamb_ndx:lamb_ndx+n_lamb], dtype=float)
        # absorbances
        self.absorb = np.zeros((self.n_times, self.n_lambdas), dtype=float)
        abs_ndx = int(_find_ndx(data, 'data:')) + 1
        data = data[abs_ndx:]
        for i in range(self.n_times):
            self.absorb[i,:] = np.array(data[i*self.n_lambdas:(i+1)*self.n_lambdas], dtype=float)

    def write(self, filename=None, format=None) -> None:
        '''
            Wrapper to write files based on extension/format

            If 'format' is not specified, the extension of the file
            is used. If the extension is not recognized, the file
            is assumed to be in the 'glb' format.

            If 'filename' is not specified, the filename is set to
            the original filename with '_t' appended to the basename.

            Supported formats: glb
        '''
        # guess format from extension if not specified
        writers = {
            'glb' : self.write_glb
            }
        # assign format based on input argument/file extension/default
        filename = filename or self.filename_trim
        extension = os.path.splitext(filename)[1][1:].lower()
        if format is not None:
            if format.lower() in writers:
                format = format.lower()
            else:
                raise ValueError(f'Unknown format to write: {format}')
        elif extension in writers:
            format = extension
        else:
            format = list(writers.keys())[0]
        # write file based on format
        writers[format](filename)

    def write_glb(self, filename) -> None:
        '''Write a spectrum to a GLB file'''
        # header
        data = "APL-ASCII-SPECTRAKINETIC\nTYPE > DATA\n%\n%\n/\n"
        data += f"N_spec: {self.n_spec:d}\nN_lam: {self.n_lambdas:d}"
        # times
        data += "\nTimes:\n"
        data += '\n'.join([f'{i:.6f}' for i in self.times])
        # wavelengths
        data += "\nLambda:\n"
        data += '\n'.join([f'{i:.3f}' for i in self.lambdas])
        # absorbances
        data += "\nData:\n"
        for i in range(self.n_times):
            data += '\n'.join([f'{j:.6f}' for j in self.absorb[i,:]])
            data += '\n\n'
        # write file
        with open(filename, 'w') as f:
            f.write(data)

    def plot(self, style='2d-times') -> None:
        '''
            Display a plot of the spectra

            Parameters
            ----------
            style : str, optional
                type of plot to draw (def: '2d-times')
                '2d-times' : multiple superposed spectra (times)
                             wavelength (x) vs. absorbance (y)
                '2d-lambdas' : multiple superposed spectra (wavelengths)
                               time (x) vs. absorbance (y)
                '3d' : 3D plot of spectra
                       time (x) vs. wavelength (y) vs. absorbance (z)
        '''
        if style == '2d-times':
            fig = plt.figure()
            ax = fig.add_subplot()
            ax.set_xlabel('Wavelength (λ)')
            ax.set_ylabel('Absorbance')
            for i in range(self.n_times):
                ax.plot(self.lambdas, self.absorb[i,:])
        elif style == '2d-lambdas':
            fig = plt.figure()
            ax = fig.add_subplot()
            ax.set_xlabel('Time')
            ax.set_ylabel('Absorbance')
            for i in range(self.n_lambdas):
                ax.plot(self.times, self.absorb[:, i])
        elif style == '3d':
            raise NotImplementedError()
        else:
            raise ValueError(f'Unknown style to plot: {style}')
        plt.show()

    def zero(self, lamb) -> None:
        '''Modify absorbances to make zero at a specific wavelength'''
        lamb_ndx = np.argmin(abs(self.lambdas-lamb))
        for i in range(self.n_times):
            self.absorb[i, :] -= self.absorb[i, lamb_ndx]

    def trim(self, time=[], lamb=[]) -> None:
        '''
            Trim spectra to a specific time/wavelength range

            Parameters
            ----------
            time : list, optional
                min and max time to keep
            lamb : list, optional
                min and max wavelength to keep
        '''
        # trim times
        if time:
            if len(time) != 2:
                raise ValueError("'time' must be a list of length 2")
            time.sort()
            time_ndx_min = np.argmin(abs(self.times-time[0]))
            time_ndx_max = np.argmin(abs(self.times-time[1]))
            self.times = self.times[time_ndx_min:time_ndx_max+1]
            self.absorb = self.absorb[time_ndx_min:time_ndx_max+1]
        # trim wavelengths
        if lamb:
            if len(lamb) != 2:
                raise ValueError("'lamb' must be a list of length 2")
            lamb.sort()
            lamb_ndx_min = np.argmin(abs(self.lambdas-lamb[0]))
            lamb_ndx_max = np.argmin(abs(self.lambdas-lamb[1]))
            self.lambdas = self.lambdas[lamb_ndx_min:lamb_ndx_max+1]
            self.absorb = self.absorb[:, lamb_ndx_min:lamb_ndx_max+1]


In [ ]:
#@title Upload spectra
#@markdown Upload a file from your local computer containing multiple spectra. Select the format of the file to be read.

format = 'glb'  # @param ["glb"]

spectrum = TDSpectrum()
file_uploaded = files.upload()
spectrum.read(list(file_uploaded.keys())[0], format=format)
spectrum.plot('2d-times')

In [ ]:
#@title Download spectra
#@markdown Write the modified spectra to a new file and download it to your computer. If the name of the file is left blank, the original name with a '_t' will be used. The format will be infered from the extension typed, the input file format or '.glb' by default.

file_name = ""  # @param {type:"string"}

file_name = file_name or spectrum.filename_trim
spectrum.write(file_name)
files.download(file_name)